# Dev Env Set

In [6]:
from dotenv import load_dotenv
import os

load_dotenv(".env", verbose=True)
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
HCC_OPENAI_API_KEY = os.getenv('HCC_OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = HCC_OPENAI_API_KEY

In [115]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY=LANGCHAIN_API_KEY
LANGCHAIN_PROJECT="kyochul"

In [116]:
import bs4
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate


# Load Documents

In [117]:
loader = PyPDFLoader('./parliamentary_records/국회회의록_22대_416회_3차_농림축산식품해양수산위원회 (1) (1).pdf')
docs = loader.load()
print(f'len(docs): {len(docs)}')
docs

len(docs): 132


[Document(metadata={'source': './parliamentary_records/국회회의록_22대_416회_3차_농림축산식품해양수산위원회 (1) (1).pdf', 'page': 0}, page_content='ਘ\x12\x17ੌ\n\x01\x01\x12\n제 416 회국회\n(임시회)농림축산식품해양수산위원회회의록\n(임시회의록)제 3 호\n국  회  사  무  처\n일   시   2024년7월16일(화)\n장   소   농림축 산식품 해양수 산위원 회회의 실\n    의사일정\n1.  업무현 황보고\n  가.  농업협 동조합 중앙회\n  나.  농협경 제지주\n  다.  농협금 융지주\n  라.  한국마 사회\n  마.  축산물 품질평 가원\n  바.  농림식 품기술 기획평 가원\n  사.  농림수 산식품 교육문 화정보 원\n  아.  농업정 책보험 금융원\n  자.  가축위 생방역 지원본 부\n  차.  축산환 경관리 원\n  카.  한국식 품산업 클러스 터진흥 원\n  타.  한식진 흥원\n  파.  한국농 업기술 진흥원\n  하.  한국임 업진흥 원\n  거.  한국산 림복지 진흥원\n  너.  한국수 목원정 원관리 원\n  더.  부산항 만공사\n  러.  인천항 만공사\n  머.  여수광 양항만 공사\n  버.  울산항 만공사\n  서.  한국해 양수산 연수원\n  어.  국립해 양생물 자원관\n  저.  국립해 양박물 관\n  처.  국립해 양과학 관\n2.  소위원  선임  및  소위원 장  선출 의  건(추 가 )\n3.  양곡관 리법  일부개 정법률 안(윤 준 병  의원  대표발 의)(의안번 호  2200017)(추가)\n4.  양곡관 리법  일부개 정법률 안(이 원 택  의원  대표발 의)(의안번 호  2200089)(추가)'),
 Document(metadata={'source': './parliamentary_records/국회회의록_22대_416회

In [118]:
docs[1].__dict__

{'id': None,
 'metadata': {'source': './parliamentary_records/국회회의록_22대_416회_3차_농림축산식품해양수산위원회 (1) (1).pdf',
  'page': 1},
 'page_content': 'ਘ\x12\x17ੌ\n5.  양곡관 리법  일부개 정법률 안(황 명 선  의원  대표발 의)(의안번 호  2200299)(추가)\n6.  양곡관 리법  일부개 정법률 안(어 기 구  의원  대표발 의)(의안번 호  2200572)(추가)\n7.  농 수산물  유통  및  가격안 정에  관한  법률  일부개 정법률안(윤준병  의원  대표발의 ) ( 의안 번 호  \n2200016)(추가)\n8.  농 수산물  유통  및  가격안 정에  관한  법률  일부개 정법률안(이원택  의원  대표발의 ) ( 의안 번 호  \n2200088)(추가)\n9.  농 수산물  유통  및  가격안 정에  관한  법률  일부개 정법률안(문금주  의원  대표발의 ) ( 의안 번 호  \n2200260)(추가)\n10.  농 수산물  유통  및  가 격안정 에  관한  법률  일부개 정법률 안( 황명선  의원  대표 발의) ( 의안번 호  \n2200301)(추가)\n11.  농 수산물  유통  및  가 격안정 에  관한  법률  일부개 정법률 안( 이병진  의원  대표 발의) ( 의안번 호  \n2200505)(추가)\n12.  농 수산물  유통  및  가 격안정 에  관한  법률  일부개 정법률 안( 어기구  의원  대표 발의) ( 의안번 호  \n2200569)(추가)\n13.  농 수산물  유통  및  가 격안정 에  관한  법률  일부개 정법률 안( 주철현  의원  대표 발의) ( 의안번 호  \n2200585)(추가)\n14.  농어업 회의소 법안(문금주  의원  대표 발의) (의안 번 호  2200520)(추가)\n15.  지속가 능한  한우 산업을  위한  지원법 안( 

# Split Docs

In [119]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

# Vectorstore

In [120]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieval

In [121]:
retriever = vectorstore.as_retriever()

# Prompt

In [122]:
template = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Just output the concise short-form answering without any additional explanation.\nQuestion: {question} \nContext: {context} \nAnswer:"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Just output the concise short-form answering without any additional explanation.\nQuestion: {question} \nContext: {context} \nAnswer:")

# Create LLM

In [123]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [124]:
retriever | format_docs

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7ff1a0c53760>)
| RunnableLambda(format_docs)

# Create Chain

In [125]:
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Rag Langchain

In [126]:
questions = [
    '2023년에 농협이 특별 상여금으로 직원들에게 얼마를 줬어?',
    '마사회에서 온라인 발매를 도입할 당시에 가장 큰 걱정이 뭐야?',
    '강호동이 생각하는 전국 하나로마트가 소비자가격을 낮추는 대신에 할인행사를 하는 이유가 뭐야?',
    '강준석이 생각하는 내년 2월부터 부산항이 직기항이 아니고 셔틀로 이용되는데, 셔틀로 운송될 때는 어떤 장단점이 있어?',
    '농협법 제159조의2가에 어떤거에 대한 정의가 나와있어?',
    '강호동은 몇만t의 쌀 소비가 목표야?',
    '마사회가 청렴도 종합평가에서 몇년만에 2등급을 받았어?',
    '농업렵동조합이 정부에 1조 6천억을 반납한 기간이 몇년부터 몇년까지야?',
    '생축장의 목적이 뭐야?',
    '소위원회에서 농림축산식품법안심사 위원은 몇명이야?'
]

In [127]:
answers = [
    '4100억',
    '불법도박',
    '과잉생산 문제 해결 및 경영적인 측면의 문제 해결',
    '15일 단축',
    '농업 지원비 부과에 대한 명확한 정의',
    '목표가 딱히 안정해져 있음',
    '굉장히 오랜만임(정확한 기간은 안나와있음)',
    '2009년부터 16년 사이',
    '한우 번식 사업을 통해서 양질의 송아지를 생산하고 저렴한 가격으로 농가에 분양하는 목적',
    '11명'
]

In [128]:
responses = []
for q in questions:
    responses.append(rag_chain.invoke(q))

In [129]:
# 결과 출력
# print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
for i in range(10):
    print(f'question: {questions[i]}, \nresponse: {responses[i]}, \nanswer: {answers[i]}')
    print()

문서의 수: 132
question: 2023년에 농협이 특별 상여금으로 직원들에게 얼마를 줬어?, 
response: 4100억원., 
answer: 4100억

question: 마사회에서 온라인 발매를 도입할 당시에 가장 큰 걱정이 뭐야?, 
response: 온라인 경마에 대한 규제와 실명제가 가장 큰 걱정이었습니다., 
answer: 불법도박

question: 강호동이 생각하는 전국 하나로마트가 소비자가격을 낮추는 대신에 할인행사를 하는 이유가 뭐야?, 
response: 농협경제 지주가 계통구매를 통해 농자재를 판매하기 때문에 농협에서 제시하는 가격이 더 비싸다., 
answer: 과잉생산 문제 해결 및 경영적인 측면의 문제 해결

question: 강준석이 생각하는 내년 2월부터 부산항이 직기항이 아니고 셔틀로 이용되는데, 셔틀로 운송될 때는 어떤 장단점이 있어?, 
response: 셔틀을 이용하면 여정이 단축되어 15일 정도 시간을 절약할 수 있습니다., 
answer: 15일 단축

question: 농협법 제159조의2가에 어떤거에 대한 정의가 나와있어?, 
response: 농협법 제159조의2에 농업지원비 부과에 대한 정의가 나와있어., 
answer: 농업 지원비 부과에 대한 명확한 정의

question: 강호동은 몇만t의 쌀 소비가 목표야?, 
response: 3000만t, 
answer: 목표가 딱히 안정해져 있음

question: 마사회가 청렴도 종합평가에서 몇년만에 2등급을 받았어?, 
response: 굉장히 오랜만에 2등급을 받았다., 
answer: 굉장히 오랜만임(정확한 기간은 안나와있음)

question: 농업렵동조합이 정부에 1조 6천억을 반납한 기간이 몇년부터 몇년까지야?, 
response: 2009년부터 16년까지, 
answer: 2009년부터 16년 사이

question: 생축장의 목적이 뭐야?, 
response: 군급식 농축산물 수의계약 유지, 농업소득 3000만원 시대로 하겠다, 한우 유통구조

# Temp

In [41]:
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_community.llms import OpenAI

In [95]:
prompt2 = "Please rate how similar are they between 1)'{response}' and 2)'{answer}'. If you think that 1) and 2) are conveying the same meaning, give 100. However, if the information they contain are not the same at all, then give 0. Just give the score without containing any other explanation."
prompt2 = PromptTemplate.from_template(prompt2)

In [96]:
chain1 = LLMChain(llm=llm, prompt=prompt, output_key='response', verbose=True)
chain2 = LLMChain(llm=llm, prompt=prompt2, verbose=True)

In [97]:
chain = SequentialChain(chains=[chain1, chain2], input_variables=['context', 'question', 'answer'], verbose=True)

In [98]:
ans = chain.run({'context': 'Kyochul is 27 years old', 'question': 'how old is Kyochul?', 'answer': '27'})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Just output the concise short-form answering without any additional explanation.
Question: how old is Kyochul? 
Context: Kyochul is 27 years old 
Answer:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Please rate how similar are they between 1)'Kyochul is 27 years old.' and 2)'27'. If you think that 1) and 2) are conveying the same meaning, give 100. However, if the information they contain are not the same at all, then give 0. Just give the score without containing any other explanation.

> Finished chain.

> Finished chain.


In [99]:
ans

'90'

In [ ]:
# 결과 출력
# print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
for i in range(10):
    print(f'question: {questions[i]}, \nresponse: {responses[i]}, \nanswer: {answers[i]}')
    ans = chain.run({'context': {, 'question': 'how old is Kyochul?', 'answer': '27'})
    print()

In [102]:
for idx, q in enumerate(questions):
    # response = rag_chain.invoke(q)
    
    ans = chain.run({'context': retriever | format_docs, 'question': q, 'answer': answers[idx]})
    print(ans)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Just output the concise short-form answering without any additional explanation.
Question: 2023년에 농협이 특별 상여금으로 직원들에게 얼마를 줬어? 
Context: first=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7ff1a0c459d0>) last=RunnableLambda(format_docs) 
Answer:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Please rate how similar are they between 1)'알 수 없어요.' and 2)'4100억'. If you think that 1) and 2) are conveying the same meaning, give 100. However, if the information they contain are not the same at all, then give 0. Just give the score without containing any other explanation.

> Finished chain.

> Finished